In [1]:
import torch
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy

import torch.nn.init as init

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())


PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Convert to tensor and scale to [0, 1]
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])
mnist_trainset = datasets.MNIST('/home/c/cl237/', train=True, download=True, transform=ts)
mnist_testset = datasets.MNIST(root='/home/c/cl237/', train=False, download=True, transform=ts)

In [3]:
from torch.utils.data import Subset
train_subset_size = int(0.1 * len(mnist_trainset))
test_subset_size = int(0.0999 * len(mnist_testset))
train_indices = list(range(len(mnist_trainset)))
test_indices = list(range(len(mnist_testset)))
torch.manual_seed(10)

seed_value = 10
np.random.seed(seed_value)

np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

train_subset_indices = train_indices[:train_subset_size]
test_subset_indices = test_indices[:test_subset_size]

mnist_trainset = Subset(mnist_trainset, train_subset_indices)
mnist_testset = Subset(mnist_testset, test_subset_indices)

In [4]:
x_d0 = mnist_trainset[0][0].size()[0]
x_d1 = mnist_trainset[0][0].size()[1]
x_d2 = mnist_trainset[0][0].size()[2]
# ([1, 28, 28])
N = x_d3 = len(mnist_trainset)
K = 10
x_train = torch.empty((N,x_d0*x_d1*x_d2), device=device)
# (60000, 28*28)

y_train = torch.empty(N, dtype=torch.long)

#torch.empty() return a tensor filled with garbage values Dimention: N by x_d0*x_d1*x_d2

for i in range(N):
     x_train[i,:] = torch.reshape(mnist_trainset[i][0], (1, x_d0*x_d1*x_d2))
     y_train[i] = mnist_trainset[i][1]
x_train = torch.t(x_train)
y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

# Manipulate test set
N_test = x_d3_test = len(mnist_testset)
x_test = torch.empty((N_test,x_d0*x_d1*x_d2), device=device)
y_test = torch.empty(N_test, dtype=torch.long)
for i in range(N_test):
     x_test[i,:] = torch.reshape(mnist_testset[i][0], (1, x_d0*x_d1*x_d2))
     y_test[i] = mnist_testset[i][1]
x_test = torch.t(x_test)
y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [5]:
#### Reshape X to X-bar
x_trainTensor = torch.reshape(x_train, (x_d1, x_d2, x_d0,-1))
x_testTensor  = torch.reshape(x_test, (x_d1, x_d2, x_d0,-1))
print(x_trainTensor.shape)
print(x_testTensor.shape)

torch.Size([28, 28, 1, 6000])
torch.Size([28, 28, 1, 999])


In [6]:
#### Chenyang: you need to write a function to transform the x_trainTensor, x_testTensor to (H'W')*(l^2C)*n
#### follow the paper in https://arxiv.org/pdf/1611.03214.pdf   this paper donot have the stride, we should also include stride
#### how to make it faster;  make sure the code is correct?
def inputX_CNN(x_Tensor, filter_size, stride):
    H, W, C, n = x_Tensor.size()
    Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
    Hprime = Hprime.to(torch.int)
    Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
    Wprime = Wprime.to(torch.int)
    Xtranform = torch.zeros((Hprime * Wprime, filter_size * filter_size * C, n), device=device)
    for i in range(n):
      Data =  x_Tensor[:,:,:,i]
      for hh in range(Hprime):
        for ww in range(Wprime):
          #  print(range(ww * stride, ww * stride + filter_size))
            DataTemp = Data[range(hh * stride, hh * stride + filter_size), :,:]
            DataTemp = DataTemp[:, range(ww * stride, ww * stride + filter_size), :]
            Xtranform[Hprime * hh + ww, :, i] = torch.reshape(DataTemp, (1, filter_size * filter_size * C))
    return Xtranform

In [7]:
### from X-bar to X-bar-bar
filter_size=3
stride = 2
x_trainTS = inputX_CNN(x_trainTensor,filter_size,stride)
x_testTS  = inputX_CNN(x_testTensor,filter_size,stride)

In [8]:
#### To-do-list
#### Chenyang: you need to write a function to update W for CNN
def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
    d,N = V.size()
    I = torch.eye(N, device=device)
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    # _, col_U = U_prime.size()
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+(alpha+tau)*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix)
    # bstar = 0
    return Wstar
# def updateWb_CNN(U, V, W, W_tensor_rec, alpha, rho,tau): # W:previous W, v for the first layer should be X-bar-bar-prime
#     W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
#     W_tensor2matrix = W_tensor_rec.reshape(W.shape)  #w shape need to take a look at
#     d,N = V.size()
#     I = torch.eye(N, device=device)
#     U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
#     A = rho*torch.mm(torch.t(V),U_prime)+alpha*W+tau*W_tensor2matrix
#     A = torch.t(A)
#     # _, col_U = U_prime.size()
#     Wstar_transpose = torch.mm(A, torch.inverse(rho*(torch.mm(torch.t(V), V))+(alpha+tau)*I))
#     Wstar =  torch.t(Wstar_transpose)
#     return Wstar

def updateWb_CNNorg(U, V, W, alpha, rho):
    d,N = V.size()
    I = torch.eye(N, device=device)
    #_, col_U = U.size()
    U_prime = torch.t(U).reshape(n*Hprime*Wprime,-1)
    Wstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(V),V))+alpha*I), rho*torch.mm(torch.t(V),U_prime)+alpha*W)
    
    return Wstar

def updateV(U1,U2,W,b,rho,gamma):
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho):
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau):
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [9]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv')
niter = 2000
rank = 220
tau = 0.2
gamma = 0.5
alpha = 1
rho = 0.5

print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)

S = 32 ### number of filters 2^5
H, W, C, n = x_trainTensor.size()   # n is the same thing as N
Hprime = torch.floor(torch.tensor((H-filter_size)/stride))+1
Hprime = Hprime.to(torch.int)
Wprime = torch.floor(torch.tensor((W-filter_size)/stride))+1
Wprime = Wprime.to(torch.int)



for Out_iter in range(1):
    rank_initial = 700
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)

    ## Chenyang
    d1 =  Hprime * Wprime * S
    d2 =  1024
    d3 =  1024
    d4 =  10


    W1 = 0.01*torch.randn(filter_size * filter_size * C, S, device=device)  ## How people usually initialize the CNN kernel?
    #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2)) ## TBD
    #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    #factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, 1))
    #W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = init.xavier_normal_(torch.empty(d2, d1, device=device))
    W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)) # 2^10 and Hprime * Wprime  *2^5*1^3
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = init.xavier_normal_(torch.empty(d3, d2, device=device))
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = init.xavier_normal_(torch.empty(d4, d3, device=device))
    b4 = 0*torch.ones(d4, 1, device=device)

    ## Chenyang: this is for the initialization of U1
    x_trainTS1 = torch.reshape(x_trainTS, (-1,n))
    x_trainTS1 = torch.t(x_trainTS1)
    x_trainTS1 = torch.reshape(x_trainTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar'
    U1prime = torch.matmul(x_trainTS1, W1)
    U1prime = torch.reshape(U1prime, (n,-1))
    U1 = torch.t(U1prime)

    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = U4

    ### to create the X-bar-bar' for test data
    x_testTS1 = torch.reshape(x_testTS, (-1, N_test))
    x_testTS1 = torch.t(x_testTS1)
    x_testTS1 = torch.reshape(x_testTS1, (-1, filter_size * filter_size * C))   ### this is X-bar-bar' test


    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

  # update for last layer
        # update V4
        V4 = (y_one_hot + gamma*U4 + alpha*V4)/(1 + gamma + alpha)

        # update U4
        U4 = (gamma*V4 + rho*(torch.mm(W4,V3) + b4.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W4, b4 = updateWb_org(U4,V3,W4,b4,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)


  # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)
        W2_torch_tensor = W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


 # update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        ## Chenyang
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, W1), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        # update U1
        U1 = relu_prox(V1,(rho*XprimeWtranspose + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        ## Chenyang: you need to use the new function updateWb_CNN
        W1 = updateWb_CNNorg(U1,x_trainTS1,W1,alpha,rho)

        # G update
        #W1_torch_tensor = W1.reshape((filter_size, filter_size, C, 2,2,2,2,2))
        #W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        #factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,1))
        #W1_tl_tensor_rec = tt_to_tensor(factors1)




  # prediction for trainning data
        ### we should use reconstructed tensnor W1 for prediction
        XprimeW = torch.reshape(torch.matmul(x_trainTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (n,-1))
        XprimeWtranspose = torch.t(XprimeW)
        a1_train = nn.ReLU()(XprimeWtranspose)
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b4.repeat(1, N), W4, a3_train), dim=0)
        # check argmax and addmm, dim=0

  #Prediction for test data
        XprimeWtest = torch.reshape(torch.matmul(x_testTS1, torch.as_tensor(W1,device=device).reshape((filter_size * filter_size * C, S)).float()), (N_test,-1))
        XprimeWtesttranspose = torch.t(XprimeWtest)
        a1_test = nn.ReLU()(XprimeWtesttranspose) ## Chenyang
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        pred_test = torch.argmax(torch.addmm(b4.repeat(1, N_test), W4, a3_test), dim=0)


    #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V4,y_one_hot,2),2).cpu().numpy()

        # Eq (5) in paper
        ## Chenyang
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(XprimeWtranspose,U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,U4,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((2*Hprime,2*Wprime,4,4,4,4,4,2,2,2)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy()
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy()

        # compute training accuracy
        correct_train = pred == y_train
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n',
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k],
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    # factors1_shape=[f.shape for f in factors1]
    # Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    total_variabels=Sum_of_variables_factors2+Sum_of_variables_factors3

    layer2_CR = Sum_of_variables_factors2/(d1*d2).item()
    layer3_CR = Sum_of_variables_factors3/(d2*d3)
    Compressedlayers_CR = total_variabels/(d1*d2+d2*d3).item()
    Compressedlayers_CR2 = (total_variabels+d3*d4)/(d1*d2+d2*d3+d3*d4).item()

    # print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(layer2_CR,layer3_CR,Compressedlayers_CR,Compressedlayers_CR2)

    #this postion to add new row into existing table
    #df=pd.read_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv')
    #new_row = {'rank':rank, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
    #           'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
    #           'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1], 'layer2_CR':layer2_CR, \
    #          'layer3_CR':layer3_CR, 'Compressedlayers_CR':Compressedlayers_CR}
    #df=df.append(new_row,ignore_index=True)
    #df.to_csv('C:/Users/Mark/Desktop/CNNmnist_hyperparameter_test.csv',index=False)


filename="XavierNormal_"+ "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCDCNN_mnist_init
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})


rank= 220 tau= 0.2 gamma= 0.5 rho= 0.5 alpha 1
Train on 6000 samples, validate on 999 samples
Repeatition 1 Epoch 1 / 2000 
 - time: 7.456631183624268 - sq_loss: 541.7925415039062 - tot_loss: 789.6235775068286 - acc: 0.30483333333333335 - val_acc: 0.2882882882882883
Repeatition 1 Epoch 2 / 2000 
 - time: 5.5635974407196045 - sq_loss: 240.79664611816406 - tot_loss: 444.74568909388034 - acc: 0.4575 - val_acc: 0.45045045045045046
Repeatition 1 Epoch 3 / 2000 
 - time: 5.4304420948028564 - sq_loss: 121.17791748046875 - tot_loss: 262.6558713197708 - acc: 0.5466666666666666 - val_acc: 0.5535535535535535
Repeatition 1 Epoch 4 / 2000 
 - time: 5.427687168121338 - sq_loss: 63.676570892333984 - tot_loss: 158.98286280892788 - acc: 0.613 - val_acc: 0.6226226226226226
Repeatition 1 Epoch 5 / 2000 
 - time: 5.412630081176758 - sq_loss: 33.986812591552734 - tot_loss: 98.97621472924948 - acc: 0.6726666666666666 - val_acc: 0.6646646646646647
Repeatition 1 Epoch 6 / 2000 
 - time: 5.438501596450806 - sq

Repeatition 1 Epoch 49 / 2000 
 - time: 5.397024869918823 - sq_loss: 0.0002196118439314887 - tot_loss: 0.5250710990367224 - acc: 0.9033333333333333 - val_acc: 0.9049049049049049
Repeatition 1 Epoch 50 / 2000 
 - time: 5.3791422843933105 - sq_loss: 0.00020848789426963776 - tot_loss: 0.5117347173480085 - acc: 0.9046666666666666 - val_acc: 0.9049049049049049
Repeatition 1 Epoch 51 / 2000 
 - time: 5.389685869216919 - sq_loss: 0.0001984408445423469 - tot_loss: 0.499076723397593 - acc: 0.9051666666666667 - val_acc: 0.9049049049049049
Repeatition 1 Epoch 52 / 2000 
 - time: 5.422981023788452 - sq_loss: 0.00018932571401819587 - tot_loss: 0.4870487681007944 - acc: 0.9056666666666666 - val_acc: 0.9069069069069069
Repeatition 1 Epoch 53 / 2000 
 - time: 5.385448217391968 - sq_loss: 0.00018103374168276787 - tot_loss: 0.4756068413029425 - acc: 0.9068333333333334 - val_acc: 0.908908908908909
Repeatition 1 Epoch 54 / 2000 
 - time: 5.380790948867798 - sq_loss: 0.00017347338143736124 - tot_loss: 0.46

Repeatition 1 Epoch 96 / 2000 
 - time: 5.3959059715271 - sq_loss: 6.725087587255985e-05 - tot_loss: 0.24153608883207198 - acc: 0.9276666666666666 - val_acc: 0.918918918918919
Repeatition 1 Epoch 97 / 2000 
 - time: 5.397138595581055 - sq_loss: 6.630107964156196e-05 - tot_loss: 0.2389043872637558 - acc: 0.9278333333333333 - val_acc: 0.918918918918919
Repeatition 1 Epoch 98 / 2000 
 - time: 5.388459205627441 - sq_loss: 6.538390152854845e-05 - tot_loss: 0.23633308523712912 - acc: 0.9283333333333333 - val_acc: 0.918918918918919
Repeatition 1 Epoch 99 / 2000 
 - time: 5.429947853088379 - sq_loss: 6.447348278015852e-05 - tot_loss: 0.23382006497995464 - acc: 0.9288333333333333 - val_acc: 0.91991991991992
Repeatition 1 Epoch 100 / 2000 
 - time: 5.4237518310546875 - sq_loss: 6.358524842653424e-05 - tot_loss: 0.23136285448854324 - acc: 0.9288333333333333 - val_acc: 0.91991991991992
Repeatition 1 Epoch 101 / 2000 
 - time: 5.40288233757019 - sq_loss: 6.272601603996009e-05 - tot_loss: 0.22895964

Repeatition 1 Epoch 143 / 2000 
 - time: 5.416651487350464 - sq_loss: 3.922757605323568e-05 - tot_loss: 0.16044850408361525 - acc: 0.942 - val_acc: 0.92992992992993
Repeatition 1 Epoch 144 / 2000 
 - time: 5.393663644790649 - sq_loss: 3.8868955016369e-05 - tot_loss: 0.15933300653778132 - acc: 0.9421666666666667 - val_acc: 0.92992992992993
Repeatition 1 Epoch 145 / 2000 
 - time: 5.4157702922821045 - sq_loss: 3.8511629099957645e-05 - tot_loss: 0.15823373159219045 - acc: 0.9425 - val_acc: 0.92992992992993
Repeatition 1 Epoch 146 / 2000 
 - time: 5.407147169113159 - sq_loss: 3.815603849943727e-05 - tot_loss: 0.1571501470549265 - acc: 0.9428333333333333 - val_acc: 0.92992992992993
Repeatition 1 Epoch 147 / 2000 
 - time: 5.373761892318726 - sq_loss: 3.7810073990840465e-05 - tot_loss: 0.15608187426842052 - acc: 0.9431666666666667 - val_acc: 0.9309309309309309
Repeatition 1 Epoch 148 / 2000 
 - time: 5.367192268371582 - sq_loss: 3.7469217204488814e-05 - tot_loss: 0.1550287535210373 - acc: 0.

Repeatition 1 Epoch 190 / 2000 
 - time: 5.403175115585327 - sq_loss: 2.7003541617887095e-05 - tot_loss: 0.12116450022513163 - acc: 0.9518333333333333 - val_acc: 0.93993993993994
Repeatition 1 Epoch 191 / 2000 
 - time: 5.388059854507446 - sq_loss: 2.6823252483154647e-05 - tot_loss: 0.12054617449721262 - acc: 0.9518333333333333 - val_acc: 0.93993993993994
Repeatition 1 Epoch 192 / 2000 
 - time: 5.412670850753784 - sq_loss: 2.66481456492329e-05 - tot_loss: 0.1199344095079141 - acc: 0.9518333333333333 - val_acc: 0.93993993993994
Repeatition 1 Epoch 193 / 2000 
 - time: 5.387277841567993 - sq_loss: 2.6471847377251834e-05 - tot_loss: 0.11932925268483814 - acc: 0.952 - val_acc: 0.9409409409409409
Repeatition 1 Epoch 194 / 2000 
 - time: 5.395901441574097 - sq_loss: 2.629689515742939e-05 - tot_loss: 0.11873051477996341 - acc: 0.9521666666666667 - val_acc: 0.9409409409409409
Repeatition 1 Epoch 195 / 2000 
 - time: 5.362330436706543 - sq_loss: 2.6124149371753447e-05 - tot_loss: 0.11813795887

...

Repeatition 1 Epoch 1784 / 2000 
 - time: 5.395653963088989 - sq_loss: 1.709733282950765e-06 - tot_loss: 0.017005228473612986 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1785 / 2000 
 - time: 5.456477165222168 - sq_loss: 1.7088831327782827e-06 - tot_loss: 0.01699675108200154 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1786 / 2000 
 - time: 5.383525133132935 - sq_loss: 1.7076494032153278e-06 - tot_loss: 0.016988277025006937 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1787 / 2000 
 - time: 5.368708610534668 - sq_loss: 1.7061889820979559e-06 - tot_loss: 0.016979818230288404 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1788 / 2000 
 - time: 5.396533727645874 - sq_loss: 1.7052922203220078e-06 - tot_loss: 0.01697136175810101 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1789 / 2000 
 - time: 5.413952350616455 - sq_loss: 1.7039051272149663e-06

Repeatition 1 Epoch 1830 / 2000 
 - time: 5.377640724182129 - sq_loss: 1.6548426629015012e-06 - tot_loss: 0.01662390265587419 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1831 / 2000 
 - time: 5.398315668106079 - sq_loss: 1.653620870456507e-06 - tot_loss: 0.016615803775289352 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1832 / 2000 
 - time: 5.414762258529663 - sq_loss: 1.652267883400782e-06 - tot_loss: 0.016607721592163217 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1833 / 2000 
 - time: 5.399797439575195 - sq_loss: 1.6510407476744149e-06 - tot_loss: 0.016599645506266827 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1834 / 2000 
 - time: 5.400457859039307 - sq_loss: 1.649355340305192e-06 - tot_loss: 0.016591569504066685 - acc: 0.9928333333333333 - val_acc: 0.975975975975976
Repeatition 1 Epoch 1835 / 2000 
 - time: 5.367203950881958 - sq_loss: 1.6474266431032447e-06 

Repeatition 1 Epoch 1878 / 2000 
 - time: 5.388086795806885 - sq_loss: 1.5986548760338337e-06 - tot_loss: 0.016243847288683357 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1879 / 2000 
 - time: 5.42802882194519 - sq_loss: 1.5981883052518242e-06 - tot_loss: 0.01623610682120216 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1880 / 2000 
 - time: 5.409335613250732 - sq_loss: 1.597189452695602e-06 - tot_loss: 0.01622838072460127 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1881 / 2000 
 - time: 5.414443731307983 - sq_loss: 1.5960200698827975e-06 - tot_loss: 0.016220657267081152 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1882 / 2000 
 - time: 5.39046573638916 - sq_loss: 1.5945372524583945e-06 - tot_loss: 0.016212946248424485 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1883 / 2000 
 - time: 5.409263372421265 - sq_loss: 1.5930622794257943e

Repeatition 1 Epoch 1923 / 2000 
 - time: 5.365946531295776 - sq_loss: 1.5491900740016717e-06 - tot_loss: 0.015902778901545386 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1924 / 2000 
 - time: 5.414345979690552 - sq_loss: 1.5483228708035313e-06 - tot_loss: 0.015895360929516756 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1925 / 2000 
 - time: 5.391210556030273 - sq_loss: 1.5478958630410489e-06 - tot_loss: 0.01588794842991774 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1926 / 2000 
 - time: 5.430845260620117 - sq_loss: 1.5470744756385102e-06 - tot_loss: 0.015880548251766415 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1927 / 2000 
 - time: 5.351177930831909 - sq_loss: 1.5460867643923848e-06 - tot_loss: 0.015873159248781122 - acc: 0.9931666666666666 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1928 / 2000 
 - time: 5.414377689361572 - sq_loss: 1.5449293186975

Repeatition 1 Epoch 1968 / 2000 
 - time: 5.373169898986816 - sq_loss: 1.502478085058101e-06 - tot_loss: 0.015576064258061706 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1969 / 2000 
 - time: 5.378952980041504 - sq_loss: 1.501628275946132e-06 - tot_loss: 0.015568965771717557 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1970 / 2000 
 - time: 5.412379503250122 - sq_loss: 1.500995267633698e-06 - tot_loss: 0.015561868441864135 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1971 / 2000 
 - time: 5.396834850311279 - sq_loss: 1.5003666931079351e-06 - tot_loss: 0.015554772026575847 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1972 / 2000 
 - time: 5.390506744384766 - sq_loss: 1.49956508721516e-06 - tot_loss: 0.015547689228833405 - acc: 0.9933333333333333 - val_acc: 0.9769769769769769
Repeatition 1 Epoch 1973 / 2000 
 - time: 5.419641017913818 - sq_loss: 1.4990840782047599e